In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numba
import math
import multiprocessing
import time
import argparse

%matplotlib qt5
os.getcwd()

'C:\\Users\\Yuan\\Desktop\\AE'

In [38]:
x = np.linspace(-10,10,1000)
y = np.exp(x) / np.sum(np.exp(x))
y2 = np.exp(x / 5) / np.sum(np.exp(x / 5))
y3 = np.exp(x / 10) / np.sum(np.exp(x / 10))
plt.figure()
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')   
ax.yaxis.set_ticks_position('left')          # 指定下边的边作为 x 轴 指定左边的边为 y 轴
ax.spines['bottom'].set_position(('data', 0))   #指定 data  设置的bottom(也就是指定的x轴)绑定到y轴的0这个点上
ax.spines['left'].set_position(('data', 0))
plt.ylim(0, 0.02)
plt.xlim(-10, 10)
plt.plot(x, y, color='b', label='Origin')
plt.plot(x, y2, color='g', label='T=5')
plt.plot(x, y3, color='purple', label='T=10')
plt.legend(loc=0)

In [39]:
data_path = 'CM-6M-o18-2020.10.17-1-60-a-waveform.txt'
amplitude = []
energy = []
duration = []
rise_time = []
rms = []
with open(data_path, "r") as f:
    f.readline()
    for _ in range(1088):
        data = f.readline().split(',')
        amplitude.append(float(data[4].strip()))
        rise_time.append(float(data[5].strip()))
        duration.append(float(data[6].strip()))
        energy.append(float(data[7].strip()))
        rms.append(float(data[8].strip()))

In [44]:
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

fig = plt.figure()
ax1 = fig.add_subplot(121)
ax1.scatter(amplitude, energy, marker='o',c='',edgecolors='g', label = 'Amplitude-Energy')
# ax2 = ax1.twinx()
# ax2.scatter(amplitude, rms, marker='o',c='',edgecolors='b', label = 'Amplitude-RMS')
ax1.set_xlabel('Amplitude /μV')
ax1.set_ylabel('Energy /aJ')
ax1.set_title("Amplitude - Energy & RMS")
ax1.set_ylim(0, pow(10, 7))
ax1.legend(loc=0)
# ax2.set_ylabel('RMS /μV')
# ax2.set_ylim(0, 50)
# ax2.legend(loc=0)


ax3 = fig.add_subplot(122)
ax3.scatter(duration, amplitude, marker='o',c='',edgecolors='purple', label = 'Duration-Amplitude')
ax3.set_xlabel('Duration /μs')
ax3.set_ylabel('Amplitude /μV')
ax3.set_title("Duration - Amplitude")
# ax3.set_ylim(0, 0.0015)
ax3.legend(loc=0)
plt.tight_layout()

<ipython-input-44-451124d16998>:6: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  ax1.scatter(amplitude, energy, marker='o',c='',edgecolors='g', label = 'Amplitude-Energy')
<ipython-input-44-451124d16998>:20: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  ax3.scatter(duration, amplitude, marker='o',c='',edgecolors='purple', label = 'Duration-Amplitude')


In [ ]:
data = ['100, 12012515.3, 1, 3.56, 29602.05, 28.0, 844.6, 0.00000002, 41.10, 707786','1000, 28015119.5, 1, 3.56, 24719.24, 12.4, 843.2, 0.00000002, 23.82, 72271','101, 12012515.3, 1, 3.56, 29602.05, 28.0, 844.6, 0.00000002, 41.10, 707786']
sorted(data, key=lambda s: int(s.split(',')[0]))

In [ ]:
s=time.time()
np.sum(pow(valid_data, 2))
e=time.time()
print(e-s)

In [ ]:
thr_dB = 25
# 20 * np.log10(5 * 11.3694478249001)
pow(10, thr_dB / 20) / 5000000

In [74]:
valid_data = [0, 1.5, 2, 1, 5]
thr_V = 1.5
counts = 0
N = len(valid_data)
for idx in range(1, N):
    if valid_data[idx - 1] <= thr_V <= valid_data[idx]:
        counts += 1
counts

3

In [73]:
file = 'C:\\Users\\Yuan\\Desktop\\AE\\waveform_558.txt'
time = []
amp = []
with open(file, "r") as f:
    f.readline()
    for i in f.readlines():
        time.append(float(i.split(',')[0]))
        amp.append(float(i.split(',')[1]))
time = np.array(time)
amp = np.array(amp)
thr_μV = 11.3694478249001

def cal_features(dataset, time_label, thr_V):
    time = time_label[-1] - time_label[0]
    valid_wave_idx = np.where(abs(dataset) >= thr_V)[0]
    start = time_label[valid_wave_idx[0]]
    end = time_label[valid_wave_idx[-1]]
    duration = end - start
    max_idx = np.argmax(abs(dataset))
    amplitude = abs(dataset[max_idx])
    rise_time = time_label[max_idx] - start
    valid_data = dataset[valid_wave_idx[0]:valid_wave_idx[-1]]
    energy = np.sum(np.multiply(pow(valid_data, 2), 0.05)) / 10000
    RMS = math.sqrt(energy / duration)
    counts = cal_counts(valid_data, thr_V)
    return valid_data, duration, amplitude, rise_time, energy, RMS, counts

def cal_counts(valid_data, thr_V):
    counts = 0
    N = len(valid_data)
    for idx in range(1, N):
        if valid_data[idx - 1] <= thr_V <= valid_data[idx]:
            counts += 1
    return counts

valid_data, duration, amplitude, rise_time, energy, RMS, counts = cal_features(amp, time, thr_μV)
print('duration(μs), amplitude, rise_time(μs), energy, RMS(μV), counts')
print(duration, amplitude, rise_time, energy, RMS, counts)

duration(μs), amplitude, rise_time(μs), energy, RMS(μV), counts
3061.6 7547.689 6.949999999999999 27103.65166788875 2.9753610881059167 1058


In [ ]:
class Preprocessing:
    def __init__(self, file_name, thr_dB):
        self.file_name = file_name
        self.thr_V = pow(10, thr_dB / 20) / 5000000
        self.counts = 1
        self.duration = 0
        self.amplitude = 0
        self.rise_time = 0
        self.energy = 0

    def skip_n_column(self, file, n=3):
        for _ in range(n):
            file.readline()

    def cal_features(self, dataset, time_label):
        valid_wave_idx = np.where(abs(dataset) >= self.thr_V)[0]
        start = time_label[valid_wave_idx[0]]
        end = time_label[valid_wave_idx[-1]]
        self.duration = end - start
        max_idx = np.argmax(abs(dataset))
        self.amplitude = dataset[max_idx]
        self.rise_time = time_label[max_idx] - start
        valid_data = dataset[valid_wave_idx[0]:valid_wave_idx[-1]]
        self.energy = np.sum(pow(valid_data, 2))

        return valid_data

    def cal_counts(self, valid_data):
        N = len(valid_data)
        for idx in range(1, N):
            if valid_data[idx - 1] <= self.thr_V <= valid_data[idx] \
                or valid_data[idx] <= self.thr_V <= valid_data[idx - 1]:
                self.counts += 1

    def main(self):
        for name in tqdm(self.file_name):
            with open(name, "r") as f:
                self.skip_n_column(f)
                sample_interval = float(f.readline()[29:])
                self.skip_n_column(f)
                points_num = int(f.readline()[36:])
                channel_num = int(f.readline().strip()[16:])
                hit_num = int(f.readline()[12:])
                time = float(f.readline()[14:])
                dataset = np.array([float(i.strip("\n")) for i in f.readlines()[1:]])
                time_label = np.linspace(time, time + sample_interval * (points_num - 1), points_num)

                # calculate the duration, amplitude, rise_time, energy and counts
                valid_data = self.cal_features(dataset, time_label)
                self.cal_counts(valid_data)
#                 print(name.split('_')[2], sample_interval, points_num, channel_num, hit_num, time)
#                 print('-' * 50)
            print(self.counts, self.duration, self.amplitude, self.rise_time, self.energy)
#             return self.counts, self.duration, self.amplitude, self.rise_time, self.energy

In [2]:
path = 'C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform'
os.chdir(path)
file_list = os.listdir(path)
file_list[:1]

['CM-6M-o18-2020.10.17-1-60-a-waveform.txt']

In [ ]:
a = Preprocessing(file_list[:100], 25)
a.main()

In [ ]:
path = 'C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform'
os.chdir(path)
file_list = os.listdir(path)
thr_dB = 25
thr_V = float('%.8f' % (pow(10, thr_dB / 20) / 5000000))
core_num = 4
each_core = int(math.ceil(len(file_list) / float(4)))  # 每一个核心需要处理的list的数目



# name = 'CM-6M-o18-2020.10.17-1-60_1_100_12012515.txt'

# for name in tqdm(file_list):


In [ ]:
if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--folder_path", type=str, default="C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform", help="path to dataset")
#     parser.add_argument("--thr_dB", type=int, default=25, help="the detection threshold")
#     parser.add_argument("--threads", type=int, default=4, help="the detection threshold")
#     opt = parser.parse_args()
#     print(opt)
    folder_path = "C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform"
    threads = 4
    thr_dB = 25
#     os.chdir(path)
    file_list = os.listdir(folder_path)
    each_core = int(math.ceil(len(file_list) / float(4)))
    result = []
    # Multithreading acceleration
    pool = multiprocessing.Pool(processes=threads)
    for i in tqdm(range(0, len(file_list), each_core)):
        result.append(pool.apply_async(Preprocessing(file_list[i:i + each_core], thr_dB), (file_list[i:i + each_core], )))
    pool.close()
    pool.join()
    